# **Notebook: Backpropagation**

This notebook runs the backpropagation algorithm on a deep neural network as described in section 7.4 of the book for the least square loss function and for one data sample. In the end, it compares the derivatives with finite difference.

After working though this notebook, you will use this notebook to extend to handle softmax output and cross-entropy loss as well as multiple data points.

The notebook is a slight modification of https://github.com/udlbook/udlbook/blob/main/Notebooks/Chap07/7_2_Backpropagation.ipynb

In [67]:
import numpy as np
import matplotlib.pyplot as plt

First let's define a neural network.  We'll just choose the weights and biases randomly for now.

In [68]:
# Set seed so we always get the same random numbers
np.random.seed(0)

# Number of hidden layers
K = 5
# Number of neurons per layer
D = 6
# Input layer dimension
D_i = 1
# Output layer dimension
D_o = 1

# Make empty lists
all_weights = [None] * (K+1)
all_biases = [None] * (K+1)

# Create input and output layer
if K==0:
    all_weights[0] = np.random.normal(size=(D_o, D_i))
else:
    all_weights[0] = np.random.normal(size=(D, D_i)) # input
    all_weights[-1] = np.random.normal(size=(D_o, D)) # output
    all_biases[0] = np.random.normal(size =(D,1)) # input bias

all_biases[-1]= np.random.normal(size =(D_o,1)) # output bias

# Create intermediate layers
for layer in range(1,K):
    all_weights[layer] = np.random.normal(size=(D,D))
    all_biases[layer] = np.random.normal(size=(D,1))

In [69]:
# Define the Rectified Linear Unit (ReLU) function
def ReLU(preactivation):
    activation = preactivation.clip(0.0)
    return activation

Now let's run our random network.  The weight matrices $\boldsymbol\Omega_{1\ldots K}$ are the entries of the list "all_weights" and the biases $\boldsymbol\beta_{1\ldots K}$ are the entries of the list "all_biases".

We know that we will need the preactivations $\mathbf{f}_{0\ldots K}$ and the activations $\mathbf{h}_{1\ldots K}$ later for the backward pass of backpropagation, so we'll store and return these as well.


In [70]:
def forward_pass(net_input, all_weights, all_biases):

    # Retrieve number of layers
    K = len(all_weights) -1

    # We'll store the pre-activations at each layer in a list "all_f"
    # and the activations in a second list "all_h".
    all_f = [None] * (K+1)
    all_h = [None] * (K+1)

    #For convenience, we'll set all_h[0] to be the input, and all_f[K] will be the output
    all_h[0] = net_input

    # Run through the layers, calculating all_f[0...K-1] and all_h[1...K]
    for layer in range(K):
        # Update preactivations and activations at this layer according to eq. 7.16 (eq. 7.17 if you use pdf version)
        # Remember to use np.matmul for matrix multiplications
        # TODO -- Replace the lines below
        # all_f[layer] = all_h[layer]
        # all_h[layer+1] = all_f[layer]

        all_f[layer] = all_h[layer] @ all_weights[layer].T + all_biases[layer].T
        # all_weights(D, D_i) all_biases[0] (D,1) net_input n_samples,D_i)
        all_h[layer+1] = ReLU(all_f[layer])

    # Compute the output from the last hidden layer
    # TODO -- Replace the line below
    # all_f[K] = np.zeros_like(all_biases[-1])
    # all_f[K] = all_biases[-1] + all_h[K] @ all_weights[-1].T
    all_f[K] = all_h[K] @ all_weights[-1].T + all_biases[-1].T

    # Retrieve the output
    net_output = all_f[K]

    return net_output, all_f, all_h

In [71]:
# Define input
n_samples= 10 # Number of samples
np.random.seed(0)
net_input = np.random.normal(0,1,size=(D_i,n_samples)) # If data points as columns
net_input = net_input.T

# Compute network output
net_output, all_f, all_h = forward_pass(net_input,all_weights, all_biases)
print("True output = %3.3f, Your answer = %3.3f"%(2.757, net_output[0,0]))

True output = 2.757, Your answer = 2.757


Now let's define a cost function. We'll just use the least squares loss function for this. We'll also write a function to compute d_cost_d_output.


In [72]:
def compute_cost(net_output, y):
    I = y.shape[0]  # Number of data points if data points as columns
    return np.sum((net_output-y) * (net_output-y))/I

def d_cost_d_output(net_output, y):
    I = y.shape[0] # Number of data points if data points as columns
    return 2*(net_output -y)/I

# Define the output we later will use for evaluating the gradients.
y = np.random.normal(0,1,size=(D_o,n_samples))
L = compute_cost(net_output, y)
print("y = ",y)
print("Cost = %3.3f"%(L))

y =  [[ 0.14404357  1.45427351  0.76103773  0.12167502  0.44386323  0.33367433
   1.49407907 -0.20515826  0.3130677  -0.85409574]]
Cost = 722.835


Now let's compute the derivatives of the network.  We already computed the forward pass.  Let's compute the backward pass.

In [73]:
# We'll need the indicator function in the backwards pass, since this is the derivative of the ReLU function.
def indicator_function(x):
    x_in = np.array(x)
    x_in[x_in>0] = 1
    x_in[x_in<=0] = 0
    return x_in

# Main backward pass routine
def backward_pass(all_weights, all_biases, all_f, all_h, y):
    # We'll store the derivatives dl_dweights and dl_dbiases in lists as well
    all_dl_dweights = [None] * (K+1)
    all_dl_dbiases = [None] * (K+1)
    # And we'll store the derivatives of the cost with respect to the activation and preactivations in lists
    all_dl_df = [None] * (K+1)
    all_dl_dh = [None] * (K+1)
    # Again for convenience we'll stick with the convention that all_h[0] is the net input and all_f[k] in the net output

    # Compute derivatives of the cost with respect to the network output
    all_dl_df[K] = np.array(d_cost_d_output(all_f[K],y))

    #########


    # Now work backwards through the network from layer K to layer 0
    # TODO Change the range to the correct order in which the layers should be procesed.
    # layer_range = range(K+1)
    layer_range = range(K,-1,-1)

    for layer in layer_range:
        # TODO Calculate the derivatives of the cost with respect to the biases at layer from all_dl_df[layer]. (eq 7.21, pdf version: eq 7.22)
        # NOTE!  To take a copy of matrix X, use Z=np.array(X)
        # REPLACE THIS LINE
        # all_dl_dbiases[layer] = np.zeros_like(all_biases[layer])
        ############
        all_dl_dbiases[layer] = np.sum(all_dl_df[layer], axis=1, keepdims=True)

        # TODO Calculate the derivatives of the cost with respect to the weights at layer from all_dl_df[layer] and all_h[layer] See Prep exercise 2.5 and also eq 7.22 in the book, (pdf version: eq 7.23)
        # Don't forget to use np.matmul
        # REPLACE THIS LINE
        # all_dl_dweights[layer] = np.zeros_like(all_weights[layer])
        # all_dl_dweights[layer] = all_dl_df[layer] @ all_h[layer].T
        all_dl_dweights[layer] = (all_dl_df[layer] @ all_h[layer].T)

        # TODO: calculate the derivatives of the cost with respect to the activations from weight and derivatives of next preactivations (second part of last line of eq 7.24, pdf version: eq 7.25)
        # REPLACE THIS LINE
        # all_dl_dh[layer] = np.zeros_like(all_h[layer])
        all_dl_dh[layer] = all_weights[layer].T @ all_dl_df[layer]

        if layer > 0:
            # TODO Calculate the derivatives of the cost with respect to the pre-activation f (use derivative of ReLu function, first part of last line of eq 7.24, pdf version: eq 7.25)
            # REPLACE THIS LINE
            # all_dl_df[layer-1] = np.zeros_like(all_f[layer-1])
            all_dl_df[layer-1] = all_dl_dh[layer] * indicator_function(all_f[layer-1])

    return all_dl_dweights, all_dl_dbiases

In [74]:
all_dl_dweights, all_dl_dbiases = backward_pass(all_weights, all_biases, all_f, all_h, y)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 6 is different from 10)

In [ ]:
np.set_printoptions(precision=3)
# Make space for derivatives computed by finite differences
all_dl_dweights_fd = [None] * (K+1)
all_dl_dbiases_fd = [None] * (K+1)

# Let's test if we have the derivatives right using finite differences
delta_fd = 0.000001

# Count number of successes
bias_success = 0
weight_success = 0

# Test the dervatives of the bias vectors
for layer in range(K+1):
    dl_dbias  = np.zeros_like(all_biases[layer])
    # For every element in the bias
    for row in range(all_biases[layer].shape[0]):
        # Take copy of biases.  We'll change one element each time
        all_biases_copy = [np.array(x) for x in all_biases]
        all_biases_copy[layer][row] += delta_fd
        network_output_1, *_ = forward_pass(net_input, all_weights, all_biases_copy)
        network_output_2, *_ = forward_pass(net_input, all_weights, all_biases)
        dl_dbias[row] = (compute_cost(network_output_1, y) - compute_cost(network_output_2,y))/delta_fd
    all_dl_dbiases_fd[layer] = np.array(dl_dbias)
    print("-----------------------------------------------")
    print("Bias %d, derivatives from backprop:"%(layer))
    print(all_dl_dbiases[layer])
    print("Bias %d, derivatives from finite differences"%(layer))
    print(all_dl_dbiases_fd[layer])
    if np.allclose(all_dl_dbiases_fd[layer],all_dl_dbiases[layer],rtol=1e-03, atol=1e-07, equal_nan=False):
        print("Success!  Derivatives match.")
        bias_success+=1
    else:
        print("Failure!  Derivatives different.")



# Test the derivatives of the weights matrices
for layer in range(K+1):
    dl_dweight  = np.zeros_like(all_weights[layer])
    # For every element in the bias
    for row in range(all_weights[layer].shape[0]):
        for col in range(all_weights[layer].shape[1]):
            # Take copy of biases. We'll change one element each time
            all_weights_copy = [np.array(x) for x in all_weights]
            all_weights_copy[layer][row][col] += delta_fd
            network_output_1, *_ = forward_pass(net_input, all_weights_copy, all_biases)
            network_output_2, *_ = forward_pass(net_input, all_weights, all_biases)
            dl_dweight[row][col] = (compute_cost(network_output_1, y) - compute_cost(network_output_2,y))/delta_fd
    all_dl_dweights_fd[layer] = np.array(dl_dweight)
    print("-----------------------------------------------")
    print("Weight %d, derivatives from backprop:"%(layer))
    print(all_dl_dweights[layer])
    print("Weight %d, derivatives from finite differences"%(layer))
    print(all_dl_dweights_fd[layer])
    if np.allclose(all_dl_dweights_fd[layer],all_dl_dweights[layer],rtol=1e-03, atol=1e-07, equal_nan=False):
        print("Success!  Derivatives match.")
        weight_success+=1
    else:
        print("Failure!  Derivatives different.")

print("-----------------------------------------------")

print("SUMMARY")
print("Bias:  %d / %d derivatives match:"%(bias_success,K+1))
print("Weight:  %d / %d derivatives match:"%(weight_success,K+1))
if bias_success+weight_success == 2*(K+1):
    print("Success!  All derivatives match.")
else:
    print("Failure!  All derivatives did not match.")